In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pickle
import os

2025-08-12 14:42:02.320144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755009722.663538      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755009722.765928      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
ratings = pd.read_csv(r"/kaggle/input/movielens-20m-dataset/rating.csv")
movies = pd.read_csv(r"/kaggle/input/enriched-dataset/curated_data (1).csv")

In [9]:
movies.head()

,movie_id,title,plot,poster_path,genres,director,top_actors,release_date,vote_average,runtime
0,2,Ariel,A Finnish man goes to the city to find a job a...,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,"['Comedy', 'Drama', 'Romance', 'Crime']",Aki Kaurismäki,"['Turo Pajala', 'Susanna Haavisto', 'Matti Pel...",1988-10-21,7.100,73
1,3,Shadows in Paradise,"Nikander, a rubbish collector and would-be ent...",/nj01hspawPof0mJmlgfjuLyJuRN.jpg,"['Comedy', 'Drama', 'Romance']",Aki Kaurismäki,"['Matti Pellonpää', 'Kati Outinen', 'Sakari Ku...",1986-10-17,7.285,74
2,5,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,['Comedy'],Robert Rodriguez,"['Tim Roth', 'Jennifer Beals', 'Antonio Bander...",1995-12-09,5.868,98
3,6,Judgment Night,"Four young friends, while taking a shortcut en...",/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,"['Action', 'Crime', 'Thriller']",Stephen Hopkins,"['Emilio Estevez', 'Cuba Gooding Jr.', 'Denis ...",1993-10-15,6.461,109
4,8,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,/7ln81BRnPR2wqxuITZxEciCe1lc.jpg,['Documentary'],Timo Novotny,[],2006-01-01,7.150,80


In [3]:
ratings

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [10]:
ratings = ratings.merge(movies[['movie_id']], 
                        left_on='movieId', right_on='movie_id', 
                        how='inner')

In [11]:
from sklearn.preprocessing import LabelEncoder

user_enc = LabelEncoder()
item_enc = LabelEncoder()

ratings['user_id_enc'] = user_enc.fit_transform(ratings['userId'])
ratings['item_id_enc'] = item_enc.fit_transform(ratings['movieId'])

num_users = ratings['user_id_enc'].nunique()
num_items = ratings['item_id_enc'].nunique()

print(f"Users: {num_users}, Items: {num_items}")

Users: 138493, Items: 16069


In [13]:
def build_ncf_model(num_users, num_items, embedding_dim=64):
    user_input = Input(shape=(1,), name='user_input')
    item_input = Input(shape=(1,), name='item_input')

    user_emb = Embedding(num_users, embedding_dim, name='user_embedding')(user_input)
    item_emb = Embedding(num_items, embedding_dim, name='item_embedding')(item_input)

    user_vec = Flatten()(user_emb)
    item_vec = Flatten()(item_emb)

    concat = Concatenate()([user_vec, item_vec])
    hidden = Dense(128, activation='relu')(concat)
    hidden = Dense(64, activation='relu')(hidden)
    output = Dense(1, activation='sigmoid')(hidden)  

    model = Model([user_input, item_input], output)
    model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])
    return model

model = build_ncf_model(num_users, num_items)
model.summary()

I0000 00:00:1755010313.836335      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1755010313.836942      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 64)     │  8,863,552 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_embedding      │ (None, 1, 64)     │  1,028,416 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 64)        │          0 │ user_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 64)        │          0 │ item_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128)       │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     16,512 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         65 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,916,801 (37.83 MB)

 Trainable params: 9,916,801 (37.83 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
y = ratings['rating'] / 5.0

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    [ratings['user_id_enc'], ratings['item_id_enc']],
    y,
    epochs=20,
    batch_size=1024,  
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/20
9525/9525 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - loss: 0.0260 - mae: 0.1233 - val_loss: 0.0312 - val_mae: 0.1373
Epoch 2/20
9525/9525 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - loss: 0.0236 - mae: 0.1169 - val_loss: 0.0315 - val_mae: 0.1379
Epoch 3/20
9525/9525 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - loss: 0.0215 - mae: 0.1111 - val_loss: 0.0325 - val_mae: 0.1412
Epoch 4/20
9525/9525 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - loss: 0.0196 - mae: 0.1057 - val_loss: 0.0330 - val_mae: 0.1422


In [17]:
os.makedirs("/kaggle/working/models", exist_ok=True)

model.save("/kaggle/working/models/ncf_model.h5")

user_embeddings = model.get_layer('user_embedding').get_weights()[0]
item_embeddings = model.get_layer('item_embedding').get_weights()[0]


with open("/kaggle/working/models/ncf_embeddings.pkl", "wb") as f:
    pickle.dump({
        'user_embeddings': user_embeddings,
        'item_embeddings': item_embeddings,
        'user_enc': user_enc,
        'item_enc': item_enc,
        'num_users': num_users,
        'num_items': num_items
    }, f)